# ***Libraries and Tools***

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import classification_report

# ***BERT for Sequence Classification***

In [ ]:
# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Tokenize and encode the text data
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Prepare labels and convert them to tensors
labels = torch.tensor(labels)

# Create TensorDataset and DataLoader
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)
train_dataloader = DataLoader(dataset, sampler=RandomSampler(dataset), batch_size=batch_size)

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=eps)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

# Define the loss function with class weights
class_weights = torch.tensor([1, 2])  # Define weights for each class
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

# Fine-tune the model
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.4f}')

# Evaluation
model.eval()
predictions = []
true_labels = []

for batch in test_dataloader:
    input_ids, attention_mask, labels = batch
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, axis=1).tolist())
        true_labels.extend(labels.tolist())

# Calculate classification report
print(classification_report(true_labels, predictions))
